In [1]:
import numpy as np
import pandas as pd
import requests
import re
import json
from os import walk
from multiprocessing.pool import ThreadPool

In [2]:
URL = "https://s3.amazonaws.com/irs-form-990/index_2014.json"

response = requests.get(URL)
with open("index_2014.json", 'wb') as f:
    f.write(response.content)

In [3]:
with open("index_2014.json") as f:
    data = json.load(f)
    data = data[list(data.keys())[0]]

In [4]:
df = pd.DataFrame.from_dict(data)
df.head(5)

,EIN,TaxPeriod,DLN,FormType,URL,OrganizationName,SubmittedOn,ObjectId,LastUpdated
0,760740805,201406,93492311012114,990EZ,https://s3.amazonaws.com/irs-form-990/20141311...,ROYAL HIGH BAND BOOSTERS,2014-12-08,201413119349201211,2016-03-21T17:23:53
1,996001533,201406,93492311012164,990EZ,https://s3.amazonaws.com/irs-form-990/20141311...,GARDEN CLUB OF HONOLULU,2014-12-08,201413119349201216,2016-03-21T17:23:53
2,510172432,201406,93492311012214,990EZ,https://s3.amazonaws.com/irs-form-990/20141311...,YANKTON AREA ARTS ASSOCIATION,2014-12-08,201413119349201221,2016-03-21T17:23:53
3,510650214,201406,93492311012664,990EZ,https://s3.amazonaws.com/irs-form-990/20141311...,ERICA ESSNER PERFORMANCE INC,2014-12-08,201413119349201266,2016-03-21T17:23:53
4,042604398,201312,93492311013014,990EZ,https://s3.amazonaws.com/irs-form-990/20141311...,LEDYARD FARM EQUINE EDUCATION FOUNDATION INC,2014-12-08,201413119349201301,2016-03-21T17:23:53


In [5]:
df.shape

(387529, 9)

In [6]:
def download_url(url):
    file_name_start = url.rfind('/') + 1
    file_name = url[file_name_start:]
    output_dir = "/home/meso/git_repo/exploring_990/data_2014/"
    file_name = output_dir + file_name
    
    r = requests.get(url, stream=True)
    if r.status_code == requests.codes.ok:
        with open(file_name, 'wb') as f:
            f.write(r.content)
    return url

urls = []
for i in range(df.shape[0]): 
    urls.append(data[i]['URL'])
    
results = ThreadPool(100).imap_unordered(download_url, urls)

In [7]:
f = []
for (dirpath, dirnames, filenames) in walk('/home/meso/git_repo/exploring_990/data_2014/'):
    f.extend(filenames)
    break

In [8]:
u = list(df['URL'])
len(u)

387529

In [9]:
file_names_list = []

for i in range(len(u)):
    file_loc = u[i].rfind("/") + 1
    file_name = u[i][file_loc:]
    file_names_list.append(file_name)

In [10]:
print(len(file_names_list), str(":"), len(f))

387529 : 387482


In [11]:
diff = list(set(file_names_list) - set(f))
len(diff)

44

In [12]:
file_loc = u[i].rfind("/") + 1
url_pre = u[0][:file_loc]

In [13]:
urls = []
for i in range(len(diff)):
    url = url_pre + diff[i]
    urls.append(url)

In [14]:
for url in urls:
    download_url(url)

In [17]:
f = []
for (dirpath, dirnames, filenames) in walk('/home/meso/git_repo/exploring_990/data_2014/'):
    f.extend(filenames)
    break

In [18]:
len(f)

387526

In [20]:
diff = list(set(file_names_list) - set(f))
len(diff)

0

In [21]:
def list_duplicates(seq):
    seen = set()
    seen_add = seen.add
    return [idx for idx, item in enumerate(seq) if item in seen or seen_add(item)]

In [23]:
print(list_duplicates(file_names_list))

[143787, 145192, 385216]


In [ ]:
dup_df = df.iloc[list_duplicates(file_names_list)]

In [ ]:
dup_ein = pd.DataFrame(dup_df['EIN'])

In [ ]:
dup_ein

In [25]:
duplicates = pd.DataFrame([])

for dup in range(dup_ein.shape[0]):
    ein = dup_ein.iloc[dup]['EIN']
    duplicates = duplicates.append(df[df['EIN'] == ein])

In [26]:
duplicates

,EIN,TaxPeriod,DLN,FormType,URL,OrganizationName,SubmittedOn,ObjectId,LastUpdated
136047,981072782,201312,93492134026824,990EZ,https://s3.amazonaws.com/irs-form-990/20142134...,COUNCIL FOR RESPONSIBLE NUTRITION INTERNATIONAL,2014-09-29,201421349349202682,2016-03-21T17:23:53
143787,981072782,201312,93492134026824,990EZ,https://s3.amazonaws.com/irs-form-990/20142134...,COUNCIL FOR RESPONSIBLE NUTRITION INTERNATIONAL,2014-10-03,201421349349202682,2016-03-21T17:23:53
138615,980413360,201309,93493227033774,990,https://s3.amazonaws.com/irs-form-990/20142227...,WEIZMANN INSTITUTE OF SCIENCE,2014-09-30,201422279349303377,2016-03-21T17:23:53
145192,980413360,201309,93493227033774,990,https://s3.amazonaws.com/irs-form-990/20142227...,WEIZMANN INSTITUTE OF SCIENCE,2014-10-08,201422279349303377,2016-03-21T17:23:53
357553,660227304,201312,93493227042464,990,https://s3.amazonaws.com/irs-form-990/20141227...,HOSPITAL DE LA CONCEPCION INC,2014-09-30,201412279349304246,2016-03-21T17:23:53
385216,660227304,201312,93493227042464,990,https://s3.amazonaws.com/irs-form-990/20141227...,HOSPITAL DE LA CONCEPCION INC,2014-10-08,201412279349304246,2016-03-21T17:23:53
